In [2]:
## Imports
import os
import numpy as np
import pandas as pd
import nibabel as nb
import bids
import warnings
warnings.filterwarnings('ignore')

# Define parameters
subjects = ['sub-001', 'sub-002', 'sub-003', 'sub-004',
            'sub-005', 'sub-006', 'sub-007', 'sub-008']
subjects_plot = ['sub-001', 'sub-002', 'sub-003', 'sub-004',
                 'sub-005', 'sub-006', 'sub-007', 'sub-008', 'group']
tasks = ['FullScreen', 'FullScreenAttendFix', 'FullScreenAttendBar']
rois = ['V1', 'V2', 'V3', 'V3AB', 'hMT+', 'LO',
        'VO', 'iIPS', 'sIPS', 'iPCS', 'sPCS', 'mPCS']

# Define folders
base_dir = '/home/mszinte/disks/meso_S/data/gaze_prf'
bids_dir = "{}".format(base_dir)
pp_dir = "{}/derivatives/pp_data".format(base_dir)

# analysis settings
best_voxels_num = 250
type_analyses = ['','_best{}'.format(best_voxels_num)]
n_permutation = 10000
cortical_mask = 'cortical'
n_ecc_bins=10
verbose = False
TR = 1.3

# Get BIDS files
bids.config.set_option('extension_initial_dot', True)
layout = bids.BIDSLayout(bids_dir, 'synthetic')


In [3]:
#for subject in subjects:
for subject in ['sub-003']:
    
    print('\n{}...'.format(subject))
    
    # define folders
    fit_dir = '{}/{}/prf/fit'.format(pp_dir, subject)
    func_dir = '{}/{}/func'.format(pp_dir, subject)
    mask_dir = '{}/{}/masks'.format(pp_dir, subject)
    tsv_dir = '{}/{}/prf/tsv'.format(pp_dir, subject)

    # load pRF threshold masks
    th_mat = nb.load('{}/{}_task-FullScreen_prf_threshold.nii.gz'.format(mask_dir,subject)).get_fdata()

    # define empty coordinates index
    x_idx, y_idx, z_idx = np.zeros_like(th_mat), np.zeros_like(th_mat), np.zeros_like(th_mat)
    for x in np.arange(th_mat.shape[0]):
        for y in np.arange(th_mat.shape[1]):
            for z in np.arange(th_mat.shape[2]):
                x_idx[x,y,z], y_idx[x,y,z], z_idx[x,y,z] = x, y, z
    
    
    # -------------------------------------------------------------------------------------------------------
    # Load fit parameters
    print('\nLoad fit parameters...')
    for attend_task, attend_task_short in zip(['','AttendFix','AttendBar'], ['', '_af', '_ab']):
        for fit_param in ['amplitude','baseline','ecc','r2','sd','x','y','theta']:
            exec("fs{}_{} = nb.load('{}/{}_task-FullScreen{}_par-{}.nii.gz').get_fdata()*th_mat".format(
                attend_task_short,fit_param,fit_dir,subject,attend_task,fit_param))
            if verbose: print("load fs{}_{}".format(attend_task_short,fit_param))

    # -------------------------------------------------------------------------------------------------------
    # Create dataframe with fit parameters
    print('\nCreate dataframe with fit parameters...')
    for roi_num, roi in enumerate(rois):
        # load roi
        lh_mat = nb.load("{}/{}_{}_L.nii.gz".format(mask_dir, roi, cortical_mask)).get_fdata()
        rh_mat = nb.load("{}/{}_{}_R.nii.gz".format(mask_dir, roi, cortical_mask)).get_fdata()
        roi_mat = lh_mat + rh_mat
        roi_mat[roi_mat==0] = np.nan

        # create dataframe
        df_roi = pd.DataFrame({'subject': [subject] * fs_r2[roi_mat==True].shape[0]})
        df_roi['roi'] = [roi] * fs_r2[roi_mat==True].shape[0]
        df_roi['vox_coord'] = np.array([x_idx[roi_mat==True],y_idx[roi_mat==True],z_idx[roi_mat==True]]).T.tolist()        

        # fullscreen parameters
        for attend_task in ['', '_ab', '_af']:
            for fit_param in ['amplitude','baseline','ecc','r2','sd','x','y','theta']:
                exec("df_roi['{}_fs{}'] = fs{}_{}[roi_mat==True]".format(fit_param, attend_task, attend_task, fit_param))
                if roi_num == 11:
                    if verbose: print("create df['{}_fs{}']".format(fit_param, attend_task))
                    exec("del fs{}_{}".format(attend_task, fit_param))
                    if verbose: print("delete fs{}_{} from working memory".format(attend_task, fit_param))

        # get rank with fs
        df_roi['rank_r2_fs'] = df_roi.groupby('roi')['r2_fs'].rank(method='max', ascending=False)

        # keep best 250
        df_roi = df_roi[(df_roi.rank_r2_fs<=best_voxels_num)]

        # across roi
        if roi_num > 0: df = pd.concat([df,df_roi], ignore_index=True)
        else: df = df_roi
    
    # -------------------------------------------------------------------------------------------------------
    # Load time series
    print('\nLoad time series...')
    attend_tasks = ['AttendFix','AttendStim']
    attend_tasks_short = ['_af','_ab']
    main_task, main_task_short = 'GazeCenterFS', 'fs'

    for attend_task, attend_task_short in zip(attend_tasks, attend_tasks_short):
        for ses,ses_short in zip(['ses-01','ses-02'],['_ses1','_ses2']):
            for run, run_short in zip(['run-1','run-2'],['_run1','_run2']):
                exec("{}{}{}{}_ts = nb.load('{}/{}_{}_task-{}{}_{}_fmriprep_dct.nii.gz').get_fdata()".format(
                    main_task_short, attend_task_short, ses_short, run_short,
                    func_dir, subject, ses, attend_task, main_task, run))
                if verbose: print("load {}{}{}{}_ts".format(main_task_short, attend_task_short, ses_short, run_short))

    # -------------------------------------------------------------------------------------------------------
    # Add timeseries to the dataframe
    print('\nAdd timeseries to the dataframe...')
    fs_r2 = nb.load('{}/{}_task-FullScreen_par-r2.nii.gz'.format(fit_dir,subject)).get_fdata()*th_mat
    for roi_num, roi in enumerate(rois):
        # load roi
        lh_mat = nb.load("{}/{}_{}_L.nii.gz".format(mask_dir, roi, cortical_mask)).get_fdata()
        rh_mat = nb.load("{}/{}_{}_R.nii.gz".format(mask_dir, roi, cortical_mask)).get_fdata()
        roi_mat = lh_mat + rh_mat
        roi_mat[roi_mat==0] = np.nan

        # create dataframe
        df_roi = pd.DataFrame({'subject': [subject] * fs_r2[roi_mat==True].shape[0]})
        df_roi['roi'] = [roi] * fs_r2[roi_mat==True].shape[0]

        # fullscreen r2 parameter
        df_roi['r2_fs'] = fs_r2[roi_mat==True]

        # get time
        df_roi['time_fs'] = [np.arange(1,fs_af_ses1_run1_ts.shape[3]+1)*TR]*np.sum(roi_mat==True)

        # get data timeseries
        main_task = 'fs'
        for attend_task in ['_ab', '_af']:
            for ses in ['_ses1','_ses2']:
                for run in ['_run1','_run2']:
                    exec("df_roi['data_{}{}{}{}'] = ({}{}{}{}_ts[roi_mat==True,:]).tolist()".format(main_task, attend_task, ses, run,
                                                                                                    main_task, attend_task, ses, run))

                if roi_num == 11:
                    if verbose: print("create df['data_{}{}{}{}']".format(main_task, attend_task, ses, run))
                    exec("del {}{}{}{}_ts".format(main_task, attend_task, ses, run))
                    if verbose: print("delete {}{}{}{}_ts from working memory".format(main_task, attend_task, ses, run))

        df_roi['rank_r2_fs'] = df_roi.groupby('roi')['r2_fs'].rank(method='max', ascending=False)

        # keep best 250
        df_roi = df_roi[(df_roi.rank_r2_fs<=best_voxels_num)]

        # across roi
        if roi_num > 0: df2 = pd.concat([df2,df_roi], ignore_index=True)
        else: df2 = df_roi

    # add to df
    df2.drop(['subject', 'roi', 'r2_fs', 'rank_r2_fs'], axis=1, inplace=True)
    df = pd.concat([df,df2], axis=1)
    del df2
    
    
    # -------------------------------------------------------------------------------------------------------
    # Load events files
    # get filenames
    AttendFix_fn = layout.get(subject=subject[4:], suffix='events', return_type='filename', task='AttendFixGazeCenterFS')
    AttendBar_fn = layout.get(subject=subject[4:], suffix='events', return_type='filename', task='AttendStimGazeCenterFS')

    for attend_task_short, attend_task in zip(['af','ab'],['AttendFix','AttendBar']):
        exec("fn = {}_fn".format(attend_task))

        for ses_run_num, ses_run in enumerate(['ses1_run1','ses1_run2','ses2_run1','ses2_run2']):
            exec("fs_{}_{}_df = pd.read_csv(fn[1], sep='\t')".format(attend_task_short, ses_run))
            exec("cols = fs_{}_{}_df.columns".format(attend_task_short, ses_run))
            for col in cols:
                if verbose:print("create fs_{}_{}_{}_mat".format(attend_task_short, ses_run, col))
                exec("fs_{attend_task_short}_{ses_run}_{col}_mat = np.array(fs_{attend_task_short}_{ses_run}_df['{col}'])".format(attend_task_short=attend_task_short, col=col, ses_run=ses_run))
            

    
    # Add events to the dataframe
    print('\nAdd events to the dataframe...')
    fs_r2 = nb.load('{}/{}_task-FullScreen_par-r2.nii.gz'.format(fit_dir,subject)).get_fdata()*th_mat
    for roi_num, roi in enumerate(rois):
        # load roi
        lh_mat = nb.load("{}/{}_{}_L.nii.gz".format(mask_dir, roi, cortical_mask)).get_fdata()
        rh_mat = nb.load("{}/{}_{}_R.nii.gz".format(mask_dir, roi, cortical_mask)).get_fdata()
        roi_mat = lh_mat + rh_mat
        roi_mat[roi_mat==0] = np.nan

        # create dataframe
        df_roi = pd.DataFrame({'subject': [subject] * fs_r2[roi_mat==True].shape[0]})
        df_roi['roi'] = [roi] * fs_r2[roi_mat==True].shape[0]

        # fullscreen r2 parameter
        df_roi['r2_fs'] = fs_r2[roi_mat==True]

        # get time
        df_roi['time_fs'] = [np.arange(1,fs_af_ses1_run1_ts.shape[3]+1)*TR]*np.sum(roi_mat==True)

        
        # add events
        for attend_task in ['af','ab']:
            for ses_run_num, ses_run in enumerate(['ses1_run1','ses1_run2','ses2_run1','ses2_run2']):
                for col in cols:
                    exec("df_roi['fs_{attend_task}_{ses_run}_{col}'] = [fs_{attend_task}_{ses_run}_{col}_mat.tolist()]*fs_r2[roi_mat==True].shape[0]".format(attend_task=attend_task, ses_run=ses_run, col=col))

                    if roi_num == 11:
                        if verbose:print("create df['fs_{}_{}_{}']".format(attend_task, ses_run, col))
                        exec("del fs_{}_{}_{}_mat".format(attend_task, ses_run, col))
                        if verbose: print("delete fs_{}_{}_{}_mat from working memory".format(attend_task, ses_run, col)) 

        # keep best 250
        df_roi['rank_r2_fs'] = df_roi.groupby('roi')['r2_fs'].rank(method='max', ascending=False)
        df_roi = df_roi[(df_roi.rank_r2_fs<=best_voxels_num)]
        
        # across roi
        if roi_num > 0: df2 = pd.concat([df2,df_roi], ignore_index=True)
        else: df2 = df_roi

    # add to df
    df2.drop(['subject', 'roi', 'r2_fs', 'rank_r2_fs'], axis=1, inplace=True)
    df = pd.concat([df,df2], axis=1)
    del df2
    
    # -------------------------------------------------------------------------------------------------------
    # Save dataframe as tsv
    df_fn = "{}/{}_res4hugo_best{}.pkl".format(tsv_dir,subject,int(best_voxels_num)) 
    print('saving {}'.format(df_fn))
    df.to_pickle(df_fn)


sub-003...

Load fit parameters...

Create dataframe with fit parameters...

Load time series...

Add timeseries to the dataframe...

Add events to the dataframe...
saving /home/mszinte/disks/meso_S/data/gaze_prf/derivatives/pp_data/sub-003/prf/tsv/sub-003_res4hugo_best250.pkl
